In [2]:
%pylab inline
import numpy as np

num_egs = 5
num_features = 2
num_classifiers = 3

X = np.zeros((num_egs,num_features))
labels = np.zeros(num_egs)
X[0][0]=10; labels[0] = 1;
X[1][0]=30; labels[1] = 1;
X[2][0]=40; labels[2] = -1;
X[3][0]=60; labels[3] = -1;
X[4][0]=90; labels[4] = 1;

X[0][1]=40; 
X[1][1]=50; 
X[2][1]=40; 
X[3][1]=50; 
X[4][1]=40; 

# X[0][0]=10; labels[0] = 1;
# X[1][0]=20; labels[1] = -1;
# X[2][0]=30; labels[2] = 1;


Populating the interactive namespace from numpy and matplotlib


In [7]:
class DecisionStump():
    def __init__(self, criteria='gini'):
        self.criteria = criteria
    def fit(X, y):
        num_egs = len(X)
        num_features = len(X[0])
        # Let's sort each feature by value initially
        # This way we don't have to sort every time
        for feature in range(num_features):
            indices = np.argsort( X[:,feature] );
            X_temp = X[indices]
            

class AdaBoost():
    def __init__(self, num_classifiers='3'):
        self.num_classifiers = num_classifiers
    
    def get_unique_feature_vals(self,X):
        num_egs = len(X)
        num_features = len(X[0])
        
        # Initialize a list of sets, there should be as many sets as number of features
        unique_feature_vals = [set() for i in range(num_features)]
        for i in range(num_egs):
            for j in range(num_features):
                unique_feature_vals[j].add(X[i][j])
        return unique_feature_vals
    
#     def fit(X, y):
        # Sort the data here
        
        
#         for feature in 
#         for t in range(self.num_classifiers):
#             print

In [8]:
clf = AdaBoost()
clf.get_unique_feature_vals(X)

[{10.0, 30.0, 40.0, 60.0, 90.0}, {40.0, 50.0}]